<a href="https://colab.research.google.com/github/alegoritma/CSE-ImageProcessing-Project/blob/main/TF_OD_API_(Optic_Disc_Detection).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Auth and Dependencies
from google.colab import auth
auth.authenticate_user()
%load_ext google.colab.data_table
%%capture
%%bash
git clone --depth 1 https://github.com/tensorflow/models
cd models/research/
# Compile protos.
protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
#@title IMPORTS

import os
import sys
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

import tensorflow as tf
import tensorflow_hub as hub

from tqdm import tqdm

import requests
import pandas as pd
import bs4 as bs
import json

tf.get_logger().setLevel('ERROR')

In [ ]:
#@title Pretrained Models
zoo_url = 'https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md'
response = requests.get(zoo_url)
sp = bs.BeautifulSoup(response.content, 'lxml')
tb = sp.find_all('table')[0] 
zoo_df = pd.read_html(str(tb),encoding='utf-8', header=0)[0]
zoo_df['href'] = [np.where(tag.has_attr('href'),tag.get('href'),"no link") for tag in tb.find_all('a')]
zoo_df

,Model name,Speed (ms),COCO mAP,Outputs,href
0,CenterNet HourGlass104 512x512,70,41.9,Boxes,http://download.tensorflow.org/models/object_d...
1,CenterNet HourGlass104 Keypoints 512x512,76,40.0/61.4,Boxes/Keypoints,http://download.tensorflow.org/models/object_d...
2,CenterNet HourGlass104 1024x1024,197,44.5,Boxes,http://download.tensorflow.org/models/object_d...
3,CenterNet HourGlass104 Keypoints 1024x1024,211,42.8/64.5,Boxes/Keypoints,http://download.tensorflow.org/models/object_d...
4,CenterNet Resnet50 V1 FPN 512x512,27,31.2,Boxes,http://download.tensorflow.org/models/object_d...
5,CenterNet Resnet50 V1 FPN Keypoints 512x512,30,29.3/50.7,Boxes/Keypoints,http://download.tensorflow.org/models/object_d...
6,CenterNet Resnet101 V1 FPN 512x512,34,34.2,Boxes,http://download.tensorflow.org/models/object_d...
7,CenterNet Resnet50 V2 512x512,27,29.5,Boxes,http://download.tensorflow.org/models/object_d...
8,CenterNet Resnet50 V2 Keypoints 512x512,30,27.6/48.2,Boxes/Keypoints,http://download.tensorflow.org/models/object_d...
9,EfficientDet D0 512x512,39,33.6,Boxes,http://download.tensorflow.org/models/object_d...


In [ ]:
PROJECT_ID = "Hidden due to privacy reasons!"
LABEL_MAP_PATH = "/content/labelmap.pbtxt"
TFRECORDS_PATTERNS = [
        f'gs://{PROJECT_ID}/.../size512x512/*.tfrec',
]
TEST_PATTERN = f'gs://{PROJECT_ID}/.../size512x512/*.tfrec'
TRAINING_DIR = "training"
MODEL_ID = 9
NUM_STEPS = 1000
WARMUP_PERCENTAGE = 0.1

In [ ]:
!mkdir $TRAINING_DIR

In [ ]:
#@title Download and extract model
def download_model(model_id):
  model_url = str(zoo_df.href[model_id])
  model_name = os.path.basename(model_url).replace(".tar.gz", "")
  model_dir = tf.keras.utils.get_file(fname=model_name,
                                      cache_subdir="/content",
                                      origin=model_url,
                                      untar=True)
  return str(model_dir)

path_to_model_dir = download_model(MODEL_ID)
path_to_model_dir

30744576/30736482 [==============================] - 0s 0us/step


'/content/efficientdet_d0_coco17_tpu-32'

In [ ]:
example_pipeline_config = '/content/models/research/object_detection/configs/tf2/ssd_efficientdet_d0_512x512_coco17_tpu-8.config'
checkpoint_path = path_to_model_dir + '/checkpoint/ckpt-0'
pipeline_config_save_path = f"/content/{TRAINING_DIR}"

In [ ]:
category_index = {
    1: {'id': 13, 'name': 'Optic Disc'}
}

In [ ]:
label_map_string = ""
nl = '\n'
for item in category_index.values():
  label_map_string += "item {\n    id: %s\n    name: '%s'\n}\n" % (item['id'], item['name'])
with open(LABEL_MAP_PATH, mode='w') as f:
  f.write(label_map_string)

In [ ]:
configs = config_util.get_configs_from_pipeline_file(example_pipeline_config)

model_config = configs['model']
model_config.ssd.freeze_batchnorm = False
model_config.ssd.num_classes = len(category_index)
model_config.ssd.anchor_generator.multiscale_anchor_generator.anchor_scale = 3.0

train_config = configs['train_config']
train_config.fine_tune_checkpoint = checkpoint_path
train_config.fine_tune_checkpoint_type = "detection" #classification if only train backbone
train_config.batch_size = 8
train_config.use_bfloat16 = False # Set this to false if you are not training on a TPU
train_config.num_steps = NUM_STEPS

train_config.data_augmentation_options[1].random_scale_crop_and_pad_to_square.scale_min = .9
train_config.data_augmentation_options[1].random_scale_crop_and_pad_to_square.scale_max = 1.2

train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.learning_rate_base= 8e-2
train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.total_steps = NUM_STEPS
train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.warmup_learning_rate= .001
train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.warmup_steps = int(NUM_STEPS * WARMUP_PERCENTAGE)
train_config.optimizer.momentum_optimizer.momentum_optimizer_value = 0.9
train_config.optimizer.use_moving_average = False

train_input_reader = configs['train_input_config']
train_input_reader.label_map_path = LABEL_MAP_PATH
train_input_reader.tf_record_input_reader.input_path[:] = TFRECORDS_PATTERNS

eval_input_reader = configs['eval_input_config']
eval_input_reader.label_map_path = LABEL_MAP_PATH
eval_input_reader.tf_record_input_reader.input_path[0] = TEST_PATTERN

# detection_model = model_builder.build(
#       model_config=model_config, is_training=True)

In [ ]:
detection_model_brl = model_builder.build(
      model_config=model_config, is_training=True)

In [ ]:
pipeline_proto = config_util.create_pipeline_proto_from_configs(configs)
config_util.save_pipeline_config(pipeline_proto, pipeline_config_save_path)

In [ ]:
new_pipeline_config_path = os.path.join(pipeline_config_save_path, "pipeline.config")

In [ ]:
new_pipeline_config_path

'/content/training/pipeline.config'

In [ ]:
%%capture
!python /content/models/research/object_detection/model_main_tf2.py \
    --model_dir=$pipeline_config_save_path \
    --pipeline_config_path=$new_pipeline_config_path \
    --record_summaries={False}

In [ ]:
!python /content/models/research/object_detection/exporter_main_v2.py \ 
        --input_type=image_tensor \
        --pipeline_config_path=/content/training/pipeline.config \
        --trained_checkpoint_dir=/content/training \
        --output_directory=/content/inference_graph

In [ ]:
cloud_path f"gs://{PROJECT_ID}/effdet0-odf-train/inference_graph"
!gsutil -m cp -r /content/inference_graph/ $cloud_path